importing libraries

In [2]:
import tenseal as ts
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

Adding noise


In [3]:

def add_laplace_noise(data, epsilon, reduction_factor):
    sensitivity = 1  # Assuming sensitivity of the data
    original_scale = sensitivity / epsilon
    reduced_scale = original_scale * reduction_factor
    noisy_data = []
    for x in data:
        l=[]
        for element in x:
            l.append(element + np.random.laplace(scale=reduced_scale))
        noisy_data.append(l)
    return noisy_data


key generation

In [4]:
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

model

In [5]:
m="heart_model.pkl"
with open(m,'rb') as f:
        model_heart=pickle.load(f)

loading data

In [6]:
X = pd.read_csv('x.csv')
Y = pd.read_csv('y.csv')

In [7]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.922713,-1.468418,-0.938515,0.364165,-0.063079,-0.417635,-1.005832,0.102904,1.435481,-0.896862,-0.649113,-0.714429,-0.512922
1,1.283063,0.681005,-0.938515,1.620649,-0.352961,-0.417635,-1.005832,-0.509323,-0.696631,1.087338,0.976352,-0.714429,-2.148873
2,-0.040403,0.681005,0.032031,3.448262,0.709940,-0.417635,-1.005832,1.983316,-0.696631,-0.896862,0.976352,0.265082,1.123029
3,0.290464,0.681005,1.002577,1.049520,-1.512490,-0.417635,0.898962,1.064975,-0.696631,0.483451,0.976352,-0.714429,-0.512922
4,0.290464,0.681005,-0.938515,0.478391,-1.048678,-0.417635,0.898962,-0.072018,-0.696631,-0.551783,-0.649113,-0.714429,-2.148873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,-0.040403,0.681005,-0.938515,-0.435415,0.381407,-0.417635,-1.005832,-1.777508,1.435481,1.001069,-0.649113,0.265082,1.123029
118,1.283063,0.681005,-0.938515,-0.663867,1.077125,-0.417635,-1.005832,0.059173,-0.696631,-0.551783,-0.649113,-0.714429,-0.512922
119,0.511041,0.681005,1.973123,0.135714,-0.816773,-0.417635,0.898962,0.540209,-0.696631,-0.206705,0.976352,1.244593,-0.512922
120,-1.474158,-1.468418,1.002577,-1.120770,0.420058,-0.417635,-1.005832,0.977514,1.435481,-0.896862,0.976352,-0.714429,-0.512922


In [8]:
Y

,target
0,1
1,1
2,0
3,1
4,1
...,...
117,0
118,1
119,0
120,1


In [9]:
x_test=np.array(X)
Y_test=np.array(Y)

In [10]:
x_test

array([[-0.92271345, -1.46841752, -0.93851463, ..., -0.64911323,
        -0.71442887, -0.51292188],
       [ 1.28306308,  0.68100522, -0.93851463, ...,  0.97635214,
        -0.71442887, -2.14887271],
       [-0.04040284,  0.68100522,  0.03203122, ...,  0.97635214,
         0.26508221,  1.12302895],
       ...,
       [ 0.5110413 ,  0.68100522,  1.97312292, ...,  0.97635214,
         1.24459328, -0.51292188],
       [-1.47415758, -1.46841752,  1.00257707, ...,  0.97635214,
        -0.71442887, -0.51292188],
       [ 0.40075247, -1.46841752,  1.00257707, ...,  0.97635214,
        -0.71442887, -0.51292188]])

In [11]:
x_test=add_laplace_noise(x_test, 1, 0.5)
x_test=np.array(x_test)

In [12]:
x_test

array([[-0.94481745, -1.44321668, -1.28035689, ..., -0.7727417 ,
        -0.42738675,  0.88363217],
       [ 2.55800033,  0.77007623, -0.89523759, ...,  1.09067586,
        -0.3426144 , -2.20417551],
       [ 0.94242631,  0.94479778,  0.90181354, ...,  0.76189816,
         0.29070257,  0.67484492],
       ...,
       [ 0.12508265,  0.74157332,  2.40272991, ...,  0.59728779,
         1.48710546,  0.91926013],
       [-3.16090618, -1.40285199,  1.56578018, ...,  1.84464868,
        -0.58575714, -1.06894106],
       [ 0.30793309, -1.92115733,  1.10019175, ...,  0.82941163,
        -0.52302759,  0.09489038]])

encrypting data

In [13]:
enc_x_test=[]
for sublist in x_test:
    enc_sub = ts.ckks_vector(context, sublist)
    enc_x_test.append(enc_sub)
enc_x_test = np.array(enc_x_test)


In [14]:
enc_x_test

array([<tenseal.tensors.ckksvector.CKKSVector object at 0x00000289A6120AF0>,
      dtype=object)

In [15]:
Y_test

array([[1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
    

In [16]:
Y_test = Y_test.flatten()
Y_test = Y_test.reshape(-1)


In [17]:
Y_test

array([1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1], dtype=int64)

In [18]:
coef = model_heart.coef_.flatten()
coef = coef.reshape(-1)


In [19]:
coef

array([-0.01585557, -0.56511634,  0.64891111, -0.13193269, -0.05848895,
       -0.0019008 ,  0.25786292,  0.35862251, -0.43554263, -0.64277946,
        0.17592834, -0.56672954, -0.50877719])

encrypting model

In [20]:
enc_coef= ts.ckks_vector(context, coef)

In [21]:
model_heart.intercept_[0]

0.16005757619019084

prediction on encrypted data

In [22]:
prediction=[]

for x in enc_x_test:
    l=model_heart.intercept_[0]
    l+=x.dot(enc_coef)
    prediction.append(l)


In [23]:
prediction

decryption

In [24]:
decrypted_op=[]
for i in range (0,len(prediction)):
    if(prediction[i].decrypt()[0]>0):
        decrypted_op.append(1)
    else:
        decrypted_op.append(0)


In [25]:
decrypted_op

[1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1]

In [26]:
accuracy = accuracy_score(Y_test, decrypted_op)

In [27]:
accuracy

0.7868852459016393

In [28]:
from sklearn.metrics import classification_report
print(classification_report(decrypted_op, Y_test))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76        54
           1       0.82      0.79      0.81        68

    accuracy                           0.79       122
   macro avg       0.78      0.79      0.78       122
weighted avg       0.79      0.79      0.79       122

